In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

#import sgmcmc code 
import models.logistic_regression.logistic_regression as lr
import samplers.sgd as sgd
import samplers.sgld as sgldcv
import samplers.sgldps as sgldcvps

from metrics import imq_KSD


In [2]:
#load in the data
file_path = "../../data/real/covtype_train.csv"
test_path = "../../data/real/covtype_test.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
N = dat_array.shape[0]
x = np.column_stack([np.ones(N), dat_array[:, 1:]])
y = dat_array[:,0]
test_data = pd.read_csv(test_path)
test_array = test_data.values[:]
N_test = test_array.shape[0]
x_test = np.column_stack([np.ones(N_test), test_array[:, 1:]])
y_test = test_array[:,0]

#set up model parameters
dim = x.shape[1] 

#priors
mu_0 = np.zeros(dim) #prior mean
lambda_0 = 10.0*np.eye(dim)  #prior covariance matrix

### Importing csv files

In [3]:
methods = ["sgld", "sgldps", "sgldcv", "sgldcvps"]
samples_csv = dict()
Niter = 10**4

for i in range(len(methods)):
    method = methods[i]
    items = glob.glob(f"./out/lrb_{method}_samples.csv")[0]
    samples_csv[method] = pd.read_csv(items).iloc[(Niter+1):].reset_index(drop=True)

In [4]:
grads_csv = dict()

#gradients for SGLD 1%
method = methods[0]
lab = glob.glob(f"./out/lrb_{method}_grads.csv")[0]
grads_csv[method] = pd.read_csv(lab).iloc[Niter:].reset_index(drop=True)

#gradients for all other methods
for i in range(1, len(methods)):
    method = methods[i]
    items = glob.glob(f"./out/lrb_{method}_grads.csv")[0]
    grads_csv[method] = pd.read_csv(items).iloc[:].reset_index(drop=True)

### KSD evaluation

In [5]:
Nrep = 10

In [6]:
for k in tqdm(range(len(methods)), desc= "Methods"):
    method = methods[k]
    idx=np.arange(Niter, 0, -200)[::-1]-1
    ksd_arr = np.zeros((idx.shape[0], Nrep))
    for j in range(Nrep):
        for i in range(idx.shape[0]):
            index = idx[i]
            samples = samples_csv[method].values[:index, (dim*(j+1)):dim*(j+2)]
            grads = grads_csv[method].values[:index, (dim*(j+1)):dim*(j+2)]
            ksd_arr[i, j] = np.log10(imq_KSD(samples, grads))
        
    ksd_arr_df = pd.DataFrame(ksd_arr)
    ksd_arr_df.to_csv(f"./out/ksdcover_{method}.csv", index=False)

Methods: 100%|██████████| 4/4 [3:15:05<00:00, 2926.48s/it]  
